In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install OPENPYXL

In [2]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import date
from zone_locations import label_zone_locations
from trackman_processing import *
import os
from pitch_outcomes_batter import PitchOutcomes
import warnings

In [3]:
#Removing the future deprecation warnings within file
##Most prevalent with 90% Max EV chunk
###If you feel this is important to keep an eye on, comment out this line
warnings.filterwarnings(action = 'ignore', category = FutureWarning)

In [4]:
data = load_data(season = 2023, path = '../Work Files/2023 Clemson Regular Season')

data = data[data['BatterTeam'] == 'CLE_TIG']

In [5]:
# Read in averages and put it in a df
average_df = pd.DataFrame()
average_df = pd.read_excel('22_top100HitterKPIs.xlsx')

top_avg_heart_swing = average_df["HeartSwing%"]
top_avg_IZ_swing = average_df["IZSwing%"]
top_avg_OZ_take = average_df["OZTake"]
top_avg_correct = average_df["Correct%"]
top_avg_maxEV = average_df["MaxEV"]
top_avg_avgEV = average_df["AvgEV"]
top_avg_HH = average_df["HH%"]
top_avg_MR = average_df["MR%"]
top_avg_FBCon = average_df["FBCon%"]
top_avg_HeartCon = average_df["HeartCon%"]
top_avg_IZCon = average_df["IZCon%"]
top_avg_NonFBCon = average_df["NonFBCon%"]
top_avg_contact = average_df["Contact%"]
top_avg_gb = average_df["GB%"]
top_avg_90 = average_df["EV90"]

In [6]:
# Create Month Column
data['Month'] = 6

for i in range(data['Date'].shape[0]):
    dateVal = str(data['Date'].iloc[i])
    try:
        data['Month'].iloc[i] = dateVal[5:7]
        if i % 100 == 0:
    except:
        data['Month'].iloc[i] = 0
        
# Create Week Column

data['Week'] = 93

for i in range(data['Date'].shape[0]):
    dateVal = str(data['Date'].iloc[i])
    try:
        year = int(dateVal[0:4])
        month = int(dateVal[5:7])
        day = int(dateVal[8:10])
        weekVal = date(year, month, day).isocalendar().week
        data['Week'].iloc[i] = '0{}'.format(weekVal)
        if i % 100 == 0:
    except:
        data['Week'].iloc[i] = ''

C:\Users\harri\AppData\Local\Temp\ipykernel_14676\69025046.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Month'].iloc[i] = dateVal[5:7]


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
0
100
200
300
400

C:\Users\harri\AppData\Local\Temp\ipykernel_14676\69025046.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Week'].iloc[i] = '0{}'.format(weekVal)



500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5500
5600
5700
5800
5900
6000
6100
6200
6300


In [7]:
# Hard Hit Percentage (HHP)
maxVelo_dict = data.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].to_dict()

def hardHitPerc(df,batterId,month):
    
    maxVelo = maxVelo_dict[batterId] # get max velo for batter
    
    # cascading threshold for hard hits: within 17% (i.e., > 95mph) if max >= 115, else within 10% if max is >= 100, else default to 90 if max in [90,100), else 85 (one guy)
    thresh = maxVelo * 0.83 if maxVelo >= 115 else maxVelo * 0.9 if maxVelo >= 100 else 90 if maxVelo >= 90 else 85
    
    # filter df
    df = df[df['BatterId'] == batterId]
    df = df[df['Month'] == month]
    
    # compute hard hits
    threshHits = df[df['ExitSpeed'] > thresh].shape[0] # counts no. of hard hits
    totalHits = data[data['ExitSpeed'].notna()].shape[0] # counts no. of hits
    return (threshHits * 100 / totalHits) # hard hit percentage

In [8]:
# useful outcome labels
outcomes = {'Swing': ['StrikeSwinging','FoulBall','InPlay'],
            'Take': ['BallCalled','BallIntentional','StrikeCalled','HitByPitch'],
            'Contact': ['FoulBall','InPlay'],
            'IZ': ['Heart','Edge'],
            'OZ': ['Chase','Waste']} 

In [9]:
# Create vectors for metric name and metric value

metricName = []
metricValue = []
gameVect = []
hitterVect = []
hitterIDVect = []
dashboardVect = []
compareToAverage = []
dateType = []
dateValue = []
teamVect = []
hitterIDVect = []

# Create month dictionary

month_dict = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun',
              '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec', '':'Error'}
# Create week dictionary
week_dict = {'': 'Error', '04': '23-29 Jan', '05': '30 Jan - 5 Feb', '06': '6-12 Feb', '07': '13-19 Feb', '08': '20-26 Feb',
             '09': '27 Feb - 5 Mar', '010': '6-12 Mar', '011': '13-19 Mar', '012': '20-26 Mar', '013':'27 Mar - 2 Apr', '014': '3-9 Apr', 
             '015': '10-16 Apr', '016': '17-23 Apr', '017': '24-30 Apr', '018': '1-7 May', '019': '8-14 May', '020': '15-21 May', 
             '022': '22-28 May'}

In [10]:
# Create ZoneType Column

data = label_zone_locations(data)

In [11]:
## Plate Decisions
# heart swing %
avg_heartSwing = top_avg_heart_swing.mean()
std_heartSwing = top_avg_heart_swing.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for value in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == value]
        heartPitches = df2[df2['ZoneType'] == 'Heart'].shape[0] # total pitches in heart
        metricName.append('HeartSwing%')
        if heartPitches == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            heartSwing = ((df2[(df2['PitchCall'].isin(outcomes['Swing'])) & (df2['ZoneType'] == 'Heart')].shape[0]) / heartPitches) * 100
            metricValue.append(heartSwing)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[value])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Plate Decisions')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (heartSwing >= (avg_heartSwing + std_heartSwing * 1.5)):
                compareToAverage.append(3)
            elif (heartSwing <= (avg_heartSwing - std_heartSwing * 1.5)):
                compareToAverage.append(0)
            elif (heartSwing <= (avg_heartSwing - std_heartSwing)):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for value in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == value]
        heartPitches = df2[df2['ZoneType'] == 'Heart'].shape[0] # total pitches in heart
        metricName.append('HeartSwing%')
        if heartPitches == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            heartSwing = ((df2[(df2['PitchCall'].isin(outcomes['Swing'])) & (df2['ZoneType'] == 'Heart')].shape[0]) / heartPitches) * 100
            metricValue.append(heartSwing)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[value])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Plate Decisions')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (heartSwing >= (avg_heartSwing + std_heartSwing * 1.5)):
                compareToAverage.append(3)
            elif (heartSwing <= (avg_heartSwing - std_heartSwing * 1.5)):
                compareToAverage.append(0)
            elif (heartSwing <= (avg_heartSwing - std_heartSwing)):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df2 = data[data['BatterId'] == bId]
    heartPitches = df2[df2['ZoneType'] == 'Heart'].shape[0] # total pitches in heart
    metricName.append('HeartSwing%')
    if heartPitches == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        heartSwing = ((df2[(df2['PitchCall'].isin(outcomes['Swing'])) & (df2['ZoneType'] == 'Heart')].shape[0]) / heartPitches) * 100
        metricValue.append(heartSwing)
        if df2['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df2['AwayTeam'].iloc[0]
            date = df2['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df2['HomeTeam'].iloc[0]
            date = df2['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df2['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Plate Decisions')
        team = df2['BatterTeam']
        teamVect.append(team.iloc[0])
        if (heartSwing >= (avg_heartSwing + std_heartSwing * 1.5)):
            compareToAverage.append(3)
        elif (heartSwing <= (avg_heartSwing - std_heartSwing * 1.5)):
            compareToAverage.append(0)
        elif (heartSwing <= (avg_heartSwing - std_heartSwing)):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [12]:
# in-zone swing %
avg_IZSwing = top_avg_IZ_swing.mean()
std_IZSwing = top_avg_IZ_swing.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        inZone = df2[(df2['ZoneType'].isin(outcomes['IZ']))].shape[0] # total pitches thrown IN the zone
        metricName.append('InZoneSwing%')
        if inZone == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            IZSwing = ((df2[(df2['PitchCall'].isin(outcomes['Swing'])) & (df2['ZoneType'].isin(outcomes['IZ']))].shape[0]) / inZone) * 100
            metricValue.append(IZSwing)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Plate Decisions')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (IZSwing >= avg_IZSwing + std_IZSwing * 1.5):
                compareToAverage.append(3)
            elif (IZSwing <= avg_IZSwing - std_IZSwing * 1.5):
                compareToAverage.append(0)
            elif (IZSwing <= avg_IZSwing - std_IZSwing):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)

                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        inZone = df2[(df2['ZoneType'].isin(outcomes['IZ']))].shape[0] # total pitches thrown IN the zone
        metricName.append('InZoneSwing%')
        if inZone == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            IZSwing = ((df2[(df2['PitchCall'].isin(outcomes['Swing'])) & (df2['ZoneType'].isin(outcomes['IZ']))].shape[0]) / inZone) * 100
            metricValue.append(IZSwing)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Plate Decisions')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (IZSwing >= avg_IZSwing + std_IZSwing * 1.5):
                compareToAverage.append(3)
            elif (IZSwing <= avg_IZSwing - std_IZSwing * 1.5):
                compareToAverage.append(0)
            elif (IZSwing <= avg_IZSwing - std_IZSwing):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]
    inZone = df[(df['ZoneType'].isin(outcomes['IZ']))].shape[0] # total pitches thrown IN the zone
    metricName.append('InZoneSwing%')                
    if inZone == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        IZSwing = ((df[(df['PitchCall'].isin(outcomes['Swing'])) & (df['ZoneType'].isin(outcomes['IZ']))].shape[0]) / inZone) * 100
        metricValue.append(IZSwing)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Plate Decisions')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (IZSwing >= avg_IZSwing + std_IZSwing * 1.5):
            compareToAverage.append(3)
        elif (IZSwing <= avg_IZSwing - std_IZSwing * 1.5):
            compareToAverage.append(0)
        elif (IZSwing <= avg_IZSwing - std_IZSwing):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [13]:
# out-of-zone take %
avg_OZTake = top_avg_OZ_take.mean()
std_OZTake = top_avg_OZ_take.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        outZone = df2[(df2['ZoneType'].isin(outcomes['OZ']))].shape[0] # total pitches thrown OUT of the zone
        metricName.append('OutZoneTake%')
        if outZone == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            OZTake = ((df2[(df2['PitchCall'].isin(outcomes['Take'])) & (df2['ZoneType'].isin(outcomes['OZ']))].shape[0]) / outZone) * 100
            metricValue.append(OZTake)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Plate Decisions')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (OZTake >= avg_OZTake + std_OZTake * 1.5):
                compareToAverage.append(3)
            elif (OZTake <= avg_OZTake - std_OZTake * 1.5):
                compareToAverage.append(0)
            elif (OZTake <= avg_OZTake - std_OZTake):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        outZone = df2[(df2['ZoneType'].isin(outcomes['OZ']))].shape[0] # total pitches thrown OUT of the zone
        metricName.append('OutZoneTake%')
        if outZone == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            OZTake = ((df2[(df2['PitchCall'].isin(outcomes['Take'])) & (df2['ZoneType'].isin(outcomes['OZ']))].shape[0]) / outZone) * 100
            metricValue.append(OZTake)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Plate Decisions')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (OZTake >= avg_OZTake + std_OZTake * 1.5):
                compareToAverage.append(3)
            elif (OZTake <= avg_OZTake - std_OZTake * 1.5):
                compareToAverage.append(0)
            elif (OZTake <= avg_OZTake - std_OZTake):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
 

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]
    outZone = df[(df['ZoneType'].isin(outcomes['OZ']))].shape[0] # total pitches thrown OUT of the zone
    metricName.append('OutZoneTake%')              
    if outZone == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        OZTake = ((df[(df['PitchCall'].isin(outcomes['Take'])) & (df['ZoneType'].isin(outcomes['OZ']))].shape[0]) / outZone) * 100
        metricValue.append(OZTake)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Plate Decisions')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (OZTake >= avg_OZTake + std_OZTake * 1.5):
            compareToAverage.append(3)
        elif (OZTake <= avg_OZTake - std_OZTake * 1.5):
            compareToAverage.append(0)
        elif (OZTake <= avg_OZTake - std_OZTake):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [14]:
avg_corrPer = top_avg_correct.mean()
std_corrPer = top_avg_correct.std()

# correct % (= iz swings + oz takes / total)
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        correctSwings = df2[(df2['PitchCall'].isin(outcomes['Swing'])) & (df2['ZoneType'].isin(outcomes['IZ']))].shape[0]
        correctTakes = df2[(df2['PitchCall'].isin(outcomes['Take'])) & (df2['ZoneType'].isin(outcomes['OZ']))].shape[0]
        metricName.append('Correct%')
        if df2.shape[0] == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            corrPer = ((correctSwings + correctTakes) / df2.shape[0]) * 100
            metricValue.append(corrPer) # note denominator is all pitches
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Plate Decisions')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (corrPer >= avg_corrPer + std_corrPer * 1.5):
                compareToAverage.append(3)
            elif (corrPer <= avg_corrPer - std_corrPer * 1.5):
                compareToAverage.append(0)
            elif (corrPer <= avg_corrPer - std_corrPer):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        correctSwings = df2[(df2['PitchCall'].isin(outcomes['Swing'])) & (df2['ZoneType'].isin(outcomes['IZ']))].shape[0]
        correctTakes = df2[(df2['PitchCall'].isin(outcomes['Take'])) & (df2['ZoneType'].isin(outcomes['OZ']))].shape[0]
        metricName.append('Correct%')
        if df2.shape[0] == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            corrPer = ((correctSwings + correctTakes) / df2.shape[0]) * 100
            metricValue.append(corrPer) # note denominator is all pitches
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Plate Decisions')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (corrPer >= avg_corrPer + std_corrPer * 1.5):
                compareToAverage.append(3)
            elif (corrPer <= avg_corrPer - std_corrPer * 1.5):
                compareToAverage.append(0)
            elif (corrPer <= avg_corrPer - std_corrPer):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]                
    correctSwings = df[(df['PitchCall'].isin(outcomes['Swing'])) & (df['ZoneType'].isin(outcomes['IZ']))].shape[0]
    correctTakes = df[(df['PitchCall'].isin(outcomes['Take'])) & (df['ZoneType'].isin(outcomes['OZ']))].shape[0]
    metricName.append('Correct%')              
    if outZone == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        corrPer = ((correctSwings + correctTakes) / df.shape[0]) * 100
        metricValue.append(corrPer) # note denominator is all pitches
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Plate Decisions')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (corrPer >= avg_corrPer + std_corrPer * 1.5):
            compareToAverage.append(3)
        elif (corrPer <= avg_corrPer - std_corrPer * 1.5):
            compareToAverage.append(0)
        elif (corrPer <= avg_corrPer - std_corrPer):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [15]:
# max EV
avg_maxEV = top_avg_maxEV.mean()
std_maxEV = top_avg_maxEV.std()


for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        metricName.append('MaxEV')
        if df2.shape[0] == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            maxEV = df2.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].mean()
            metricValue.append(maxEV)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (maxEV >= avg_maxEV + std_maxEV * 1.5):
                compareToAverage.append(3)
            elif (maxEV <= avg_maxEV - std_maxEV * 1.5):
                compareToAverage.append(0)
            elif (maxEV <= avg_maxEV - std_maxEV):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        metricName.append('MaxEV')
        if df2.shape[0] == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            maxEV = df2.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].mean()
            metricValue.append(maxEV)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (maxEV >= avg_maxEV + std_maxEV * 1.5):
                compareToAverage.append(3)
            elif (maxEV <= avg_maxEV - std_maxEV * 1.5):
                compareToAverage.append(0)
            elif (maxEV <= avg_maxEV - std_maxEV):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]  
    df = df[df['PitchCall'] == 'InPlay']
    metricName.append('MaxEV')
    if df.shape[0] == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        maxEV = df.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].mean()
        metricValue.append(maxEV)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (maxEV >= avg_maxEV + std_maxEV * 1.5):
            compareToAverage.append(3)
        elif (maxEV <= avg_maxEV - std_maxEV * 1.5):
            compareToAverage.append(0)
        elif (maxEV <= avg_maxEV - std_maxEV):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

# avg EV
avg_avgEV = top_avg_avgEV.mean()
std_avgEV = top_avg_avgEV.std()

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        metricName.append('AvgEV')
        if df2.shape[0] == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            avgEV = df2['ExitSpeed'].mean()
            metricValue.append(avgEV)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (avgEV >= avg_avgEV + std_avgEV * 1.5):
                compareToAverage.append(3)
            elif (avgEV <= avg_avgEV - std_avgEV * 1.5):
                compareToAverage.append(0)
            elif (avgEV <= avg_avgEV - std_avgEV):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        metricName.append('AvgEV')
        if df2.shape[0] == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            avgEV = df2['ExitSpeed'].mean()
            metricValue.append(avgEV)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (avgEV >= avg_avgEV + std_avgEV * 1.5):
                compareToAverage.append(3)
            elif (avgEV <= avg_avgEV - std_avgEV * 1.5):
                compareToAverage.append(0)
            elif (avgEV <= avg_avgEV - std_avgEV):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    df = df[df['PitchCall'] == 'InPlay']
    metricName.append('AvgEV')             
    if df.shape[0] == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        avgEV = df['ExitSpeed'].mean()
        metricValue.append(avgEV)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (avgEV >= avg_avgEV + std_avgEV * 1.5):
            compareToAverage.append(3)
        elif (avgEV <= avg_avgEV - std_avgEV * 1.5):
            compareToAverage.append(0)
        elif (avgEV <= avg_avgEV - std_avgEV):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [16]:
# hard hit%
# Find max velo across dataset for each hitter
maxVelo_dict = data.groupby('BatterId').max('ExitSpeed')['ExitSpeed'].to_dict()

avg_hardHit = top_avg_HH.mean()
std_hardHit = top_avg_HH.std()

# Calculating Hard Hit% by Game
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    # Finding max velo for entire season
    maxVelo = maxVelo_dict[bId]
    # cascading threshold for hard hits: within 17% (i.e., > 95mph) if max >= 115, else within 10% if max is >= 100, else default to 90 if max in [90,100), else 85 (one guy)
    thresh = maxVelo * 0.83 if maxVelo >= 115 else maxVelo * 0.9 if maxVelo >= 100 else 90 if maxVelo >= 90 else 85
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        threshHits = df2[df2['ExitSpeed'] > thresh].shape[0] # counts no. of hard hits
        totalHits = df2[df2['ExitSpeed'].notna()].shape[0] # counts no. of hits
        metricName.append('HardHit%')
        if totalHits == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            hardHitPercent = (threshHits * 100 / totalHits)
            metricValue.append(hardHitPercent) # hard hit percentage
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (hardHitPercent >= avg_hardHit + std_hardHit * 1.5):
                compareToAverage.append(3)
            elif (hardHitPercent <= avg_hardHit - std_hardHit * 1.5):
                compareToAverage.append(0)
            elif (hardHitPercent <= avg_hardHit - std_hardHit):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
# Calculating Hard Hit% by Game
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    # Finding max velo for entire season
    maxVelo = maxVelo_dict[bId]
    # cascading threshold for hard hits: within 17% (i.e., > 95mph) if max >= 115, else within 10% if max is >= 100, else default to 90 if max in [90,100), else 85 (one guy)
    thresh = maxVelo * 0.83 if maxVelo >= 115 else maxVelo * 0.9 if maxVelo >= 100 else 90 if maxVelo >= 90 else 85
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        threshHits = df2[df2['ExitSpeed'] > thresh].shape[0] # counts no. of hard hits
        totalHits = df2[df2['ExitSpeed'].notna()].shape[0] # counts no. of hits
        metricName.append('HardHit%')
        if totalHits == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            hardHitPercent = (threshHits * 100 / totalHits)
            metricValue.append(hardHitPercent) # hard hit percentage
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (hardHitPercent >= avg_hardHit + std_hardHit * 1.5):
                compareToAverage.append(3)
            elif (hardHitPercent <= avg_hardHit - std_hardHit * 1.5):
                compareToAverage.append(0)
            elif (hardHitPercent <= avg_hardHit - std_hardHit):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    # Finding max velo for entire season
    maxVelo = maxVelo_dict[bId]
    # cascading threshold for hard hits: within 17% (i.e., > 95mph) if max >= 115, else within 10% if max is >= 100, else default to 90 if max in [90,100), else 85 (one guy)
    thresh = maxVelo * 0.83 if maxVelo >= 115 else maxVelo * 0.9 if maxVelo >= 100 else 90 if maxVelo >= 90 else 85    
    df = data[data['BatterId'] == bId]
    df = df[df['PitchCall'] == 'InPlay']
    threshHits = df[df['ExitSpeed'] > thresh].shape[0] # counts no. of hard hits
    totalHits = df[df['ExitSpeed'].notna()].shape[0] # counts no. of hits
    metricName.append('HardHit%')          
    if totalHits == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        hardHitPercent = (threshHits * 100 / totalHits)
        metricValue.append(hardHitPercent) # hard hit percentage
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (hardHitPercent >= avg_hardHit + std_hardHit * 1.5):
            compareToAverage.append(3)
        elif (hardHitPercent <= avg_hardHit - std_hardHit * 1.5):
            compareToAverage.append(0)
        elif (hardHitPercent <= avg_hardHit - std_hardHit):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [17]:
avg_fbCon = top_avg_FBCon.mean()
std_fbCon = top_avg_FBCon.std()


# fb contact %
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        FBs = df2[df2['TaggedPitchType'] == 'Fastball'].shape[0] # total fastballs
        fbContact = df2[(df2['TaggedPitchType'] == 'Fastball') & (df2['PitchCall'].isin(outcomes['Contact']))].shape[0]
        metricName.append('FB Contact%')
        if FBs == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            fbCon = (fbContact / FBs) * 100
            metricValue.append(fbCon)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (fbCon >= avg_fbCon + std_fbCon * 1.5):
                compareToAverage.append(3)
            elif (fbCon <= avg_fbCon - std_fbCon * 1.5):
                compareToAverage.append(0)
            elif (fbCon <= avg_fbCon - std_fbCon):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        FBs = df2[df2['TaggedPitchType'] == 'Fastball'].shape[0] # total fastballs
        fbContact = df2[(df2['TaggedPitchType'] == 'Fastball') & (df2['PitchCall'].isin(outcomes['Contact']))].shape[0]
        metricName.append('FB Contact%')
        if FBs == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            fbCon = (fbContact / FBs) * 100
            metricValue.append(fbCon)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (fbCon >= avg_fbCon + std_fbCon * 1.5):
                compareToAverage.append(3)
            elif (fbCon <= avg_fbCon - std_fbCon * 1.5):
                compareToAverage.append(0)
            elif (fbCon <= avg_fbCon - std_fbCon):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                          
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]      
    FBs = df[df['TaggedPitchType'] == 'Fastball'].shape[0] # total fastballs
    fbContact = df[(df['TaggedPitchType'] == 'Fastball') & (df['PitchCall'].isin(outcomes['Contact']))].shape[0]
    metricName.append('FB Contact%')           
    if FBs == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        fbCon = (fbContact / FBs) * 100
        metricValue.append(fbCon)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (fbCon >= avg_fbCon + std_fbCon * 1.5):
            compareToAverage.append(3)
        elif (fbCon <= avg_fbCon - std_fbCon * 1.5):
            compareToAverage.append(0)
        elif (fbCon <= avg_fbCon - std_fbCon):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [18]:
avg_heartCon = top_avg_HeartCon.mean()
std_heartCon = top_avg_HeartCon.std()

# heart contact %
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        heartContact = df2[(df2['ZoneType'] == 'Heart') & (df2['PitchCall'].isin(outcomes['Contact']))].shape[0]
        heartPitches = df2[(df2['ZoneType'] == 'Heart')].shape[0]
        metricName.append('Heart Contact%')
        if heartPitches == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            heartCon = (heartContact / heartPitches) * 100
            metricValue.append(heartCon)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (heartCon >= avg_heartCon + std_heartCon * 1.5):
                compareToAverage.append(3)
            elif (heartCon <= avg_heartCon - std_heartCon * 1.5):
                compareToAverage.append(0)
            elif (heartCon <= avg_heartCon - std_heartCon):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        heartContact = df2[(df2['ZoneType'] == 'Heart') & (df2['PitchCall'].isin(outcomes['Contact']))].shape[0]
        heartPitches = df2[(df2['ZoneType'] == 'Heart')].shape[0]
        metricName.append('Heart Contact%')
        if heartPitches == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            heartCon = (heartContact / heartPitches) * 100
            metricValue.append(heartCon)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (heartCon >= avg_heartCon + std_heartCon * 1.5):
                compareToAverage.append(3)
            elif (heartCon <= avg_heartCon - std_heartCon * 1.5):
                compareToAverage.append(0)
            elif (heartCon <= avg_heartCon - std_heartCon):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]      
    heartContact = df[(df['ZoneType'] == 'Heart') & (df['PitchCall'].isin(outcomes['Contact']))].shape[0]
    heartPitches = df[(df['ZoneType'] == 'Heart')].shape[0]
    metricName.append('Heart Contact%')        
    if heartPitches == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        heartCon = (heartContact / heartPitches) * 100
        metricValue.append(heartCon)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (heartCon >= avg_heartCon + std_heartCon * 1.5):
            compareToAverage.append(3)
        elif (heartCon <= avg_heartCon - std_heartCon * 1.5):
            compareToAverage.append(0)
        elif (heartCon <= avg_heartCon - std_heartCon):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

avg_IZCon = top_avg_IZCon.mean()
std_IZCon = top_avg_IZCon.std()

# in-zone contact %
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        zoneContact = ((df2[(df2['PitchCall'].isin(outcomes['Contact'])) & (df2['ZoneType'].isin(outcomes['IZ']))].shape[0]))
        inZone = df2[(df2['ZoneType'].isin(outcomes['IZ']))].shape[0]
        metricName.append('InZone Contact%')
        if inZone == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            IZCon = zoneContact * 100 / inZone
            metricValue.append(IZCon)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (IZCon >= avg_IZCon + std_IZCon * 1.5):
                compareToAverage.append(3)
            elif (IZCon <= avg_IZCon - std_IZCon * 1.5):
                compareToAverage.append(0)
            elif (IZCon <= avg_IZCon - std_IZCon):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)


for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        zoneContact = ((df2[(df2['PitchCall'].isin(outcomes['Contact'])) & (df2['ZoneType'].isin(outcomes['IZ']))].shape[0]))
        inZone = df2[(df2['ZoneType'].isin(outcomes['IZ']))].shape[0]
        metricName.append('InZone Contact%')
        if inZone == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            IZCon = zoneContact * 100 / inZone
            metricValue.append(IZCon)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (IZCon >= avg_IZCon + std_IZCon * 1.5):
                compareToAverage.append(3)
            elif (IZCon <= avg_IZCon - std_IZCon * 1.5):
                compareToAverage.append(0)
            elif (IZCon <= avg_IZCon - std_IZCon):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]      
    zoneContact = ((df[(df['PitchCall'].isin(outcomes['Contact'])) & (df['ZoneType'].isin(outcomes['IZ']))].shape[0]))
    inZone = df[(df['ZoneType'].isin(outcomes['IZ']))].shape[0]
    metricName.append('InZone Contact%')       
    if inZone == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        IZCon = zoneContact * 100 / inZone
        metricValue.append(IZCon)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (IZCon >= avg_IZCon + std_IZCon * 1.5):
            compareToAverage.append(3)
        elif (IZCon <= avg_IZCon - std_IZCon * 1.5):
            compareToAverage.append(0)
        elif (IZCon <= avg_IZCon - std_IZCon):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [19]:
avg_MR = top_avg_MR.mean()
std_MR = top_avg_MR.std()

# MR %
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        mr = df2[(df2['Angle'] >= 10) & (df2['Angle'] <= 30) & (df2['ExitSpeed'] > 95)].shape[0]
        contact = df2[df2['PitchCall'].isin(outcomes['Contact'])].shape[0]
        metricName.append('MR%')
        if contact == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            mr_percent = (mr / contact) * 100
            metricValue.append(mr_percent)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (mr_percent >= avg_MR + std_MR * 1.5):
                compareToAverage.append(3)
            elif (mr_percent <= avg_MR - std_MR * 1.5):
                compareToAverage.append(0)
            elif (mr_percent <= avg_MR - std_MR):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        mr = df2[(df2['Angle'] >= 10) & (df2['Angle'] <= 30) & (df2['ExitSpeed'] > 95)].shape[0]
        contact = df2[df2['PitchCall'].isin(outcomes['Contact'])].shape[0]
        metricName.append('MR%')
        if contact == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            mr_percent = (mr / contact) * 100
            metricValue.append(mr_percent)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (mr_percent >= avg_MR + std_MR * 1.5):
                compareToAverage.append(3)
            elif (mr_percent <= avg_MR - std_MR * 1.5):
                compareToAverage.append(0)
            elif (mr_percent <= avg_MR - std_MR):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)

                
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId] 
    df = df[df['PitchCall'] == 'InPlay']
    mr = df[(df['Angle'] >= 10) & (df['Angle'] <= 30) & (df['ExitSpeed'] > 95)].shape[0]
    contact = df[df['PitchCall'].isin(outcomes['Contact'])].shape[0]
    metricName.append('MR%')    
    if contact == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        mr_percent = (mr / contact) * 100
        metricValue.append(mr_percent)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (mr_percent >= avg_MR + std_MR * 1.5):
            compareToAverage.append(3)
        elif (mr_percent <= avg_MR - std_MR * 1.5):
            compareToAverage.append(0)
        elif (mr_percent <= avg_MR - std_MR):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [20]:
avg_nonFBCon = top_avg_NonFBCon.mean()
std_nonFBCon = top_avg_NonFBCon.std()

# non-fb contact %
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        nonFBs = df2[df2['TaggedPitchType'] != 'Fastball'].shape[0] # total NON fastballs
        nonFBContact = df2[(df2['TaggedPitchType'] != 'Fastball') & (df2['PitchCall'].isin(outcomes['Contact']))].shape[0]
        metricName.append('Non-FB Contact%')
        if nonFBs == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            nonFBConPer = (nonFBContact / nonFBs) * 100
            metricValue.append(nonFBConPer)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (nonFBConPer >= avg_nonFBCon + std_nonFBCon * 1.5):
                compareToAverage.append(3)
            elif (nonFBConPer <= avg_nonFBCon - std_nonFBCon * 1.5):
                compareToAverage.append(0)
            elif (nonFBConPer <= avg_nonFBCon - std_nonFBCon):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        nonFBs = df2[df2['TaggedPitchType'] != 'Fastball'].shape[0] # total NON fastballs
        nonFBContact = df2[(df2['TaggedPitchType'] != 'Fastball') & (df2['PitchCall'].isin(outcomes['Contact']))].shape[0]
        metricName.append('Non-FB Contact%')
        if nonFBs == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            nonFBConPer = (nonFBContact / nonFBs) * 100
            metricValue.append(nonFBConPer)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (nonFBConPer >= avg_nonFBCon + std_nonFBCon * 1.5):
                compareToAverage.append(3)
            elif (nonFBConPer <= avg_nonFBCon - std_nonFBCon * 1.5):
                compareToAverage.append(0)
            elif (nonFBConPer <= avg_nonFBCon - std_nonFBCon):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]      
    nonFBs = df[df['TaggedPitchType'] != 'Fastball'].shape[0] # total NON fastballs
    nonFBContact = df[(df['TaggedPitchType'] != 'Fastball') & (df['PitchCall'].isin(outcomes['Contact']))].shape[0]
    metricName.append('Non-FB Contact%')
    if nonFBs == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        nonFBConPer = (nonFBContact / nonFBs) * 100
        metricValue.append(nonFBConPer)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (nonFBConPer >= avg_nonFBCon + std_nonFBCon * 1.5):
            compareToAverage.append(3)
        elif (nonFBConPer <= avg_nonFBCon - std_nonFBCon * 1.5):
            compareToAverage.append(0)
        elif (nonFBConPer <= avg_nonFBCon - std_nonFBCon):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [21]:
avg_con = top_avg_contact.mean()
std_con = top_avg_contact.std()


# All Contact%
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        contact = df2[df2['PitchCall'].isin(outcomes['Contact'])].shape[0]
        swings = df2[df2['PitchCall'].isin(outcomes['Swing'])].shape[0]
        metricName.append('Contact%')
        if swings == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            con_per = (contact / swings) * 100
            metricValue.append(con_per)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (con_per >= avg_con + std_con * 1.5):
                compareToAverage.append(3)
            elif (con_per <= avg_con - std_con * 1.5):
                compareToAverage.append(0)
            elif (con_per <= avg_con - std_con):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        contact = df2[df2['PitchCall'].isin(outcomes['Contact'])].shape[0]
        swings = df2[df2['PitchCall'].isin(outcomes['Swing'])].shape[0]
        metricName.append('Contact%')
        if swings == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            con_per = (contact / swings) * 100
            metricValue.append(con_per)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (con_per >= avg_con + std_con * 1.5):
                compareToAverage.append(3)
            elif (con_per <= avg_con - std_con * 1.5):
                compareToAverage.append(0)
            elif (con_per <= avg_con - std_con):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]      
    contact = df[df['PitchCall'].isin(outcomes['Contact'])].shape[0]
    swings = df[df['PitchCall'].isin(outcomes['Swing'])].shape[0]
    metricName.append('Contact%')
    if swings == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        con_per = (contact / swings) * 100
        metricValue.append(con_per)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (con_per >= avg_con + std_con * 1.5):
            compareToAverage.append(3)
        elif (con_per <= avg_con - std_con * 1.5):
            compareToAverage.append(0)
        elif (con_per <= avg_con - std_con):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [22]:
avg_gb = top_avg_gb.mean()
std_gb = top_avg_gb.std()


# All Contact%
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        inPlay = df2[df2['PitchCall'] == 'InPlay'].shape[0]
        inPlayDf = df2[df2['PitchCall'] == 'InPlay']
        gbs = inPlayDf[inPlayDf['Angle'] < 0].shape[0]
        metricName.append('GB%')
        if inPlay == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            gb_per = (gbs / inPlay) * 100
            metricValue.append(gb_per)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (gb_per >= avg_gb + std_gb * 1.5):
                compareToAverage.append(0)
            elif (gb_per <= avg_gb - std_gb * 1.5):
                compareToAverage.append(3)
            elif (gb_per <= avg_gb - std_gb):
                compareToAverage.append(2)
            else:
                compareToAverage.append(1)
                
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        inPlay = df2[df2['PitchCall'] == 'InPlay'].shape[0]
        inPlayDf = df2[df2['PitchCall'] == 'InPlay']
        gbs = inPlayDf[inPlayDf['Angle'] < 0].shape[0]
        metricName.append('GB%')
        if inPlay == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            gb_per = (gbs / inPlay) * 100
            metricValue.append(gb_per)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Consistent Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (gb_per >= avg_gb + std_gb * 1.5):
                compareToAverage.append(0)
            elif (gb_per <= avg_gb - std_gb * 1.5):
                compareToAverage.append(3)
            elif (gb_per <= avg_gb - std_gb):
                compareToAverage.append(2)
            else:
                compareToAverage.append(1)
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]      
    inPlay = df[df['PitchCall'] == 'InPlay'].shape[0]
    inPlayDf = df[df['PitchCall'] == 'InPlay']
    gbs = inPlayDf[inPlayDf['Angle'] < 0].shape[0]
    metricName.append('GB%')
    if inPlay == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        gb_per = (gbs / inPlay) * 100
        metricValue.append(gb_per)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Consistent Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (gb_per >= avg_gb + std_gb * 1.5):
            compareToAverage.append(0)
        elif (gb_per <= avg_gb - std_gb * 1.5):
            compareToAverage.append(3)
        elif (gb_per <= avg_gb - std_gb):
            compareToAverage.append(2)
        else:
            compareToAverage.append(1)

In [23]:
# 90% percentile Max EV
avg_90 = top_avg_90.mean()
std_90 = top_avg_90.std()
                
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for week in week_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Week'] == week]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        metricName.append('90% EV')
        if df2.shape[0] == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            maxEV = df2.groupby('BatterId').quantile(0.9)['ExitSpeed'].mean()
            metricValue.append(maxEV)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Week')
            dateValue.append(week_dict[week])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (maxEV >= avg_90 + std_90 * 1.5):
                compareToAverage.append(3)
            elif (maxEV <= avg_90 - std_90 * 1.5):
                compareToAverage.append(0)
            elif (maxEV <= avg_90 - std_90):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
            
            
for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    for month in month_dict.keys():
        df = data[data['BatterId'] == bId]
        df2 = df[df['Month'] == month]
        df2 = df2[df2['PitchCall'] == 'InPlay']
        metricName.append('90% EV')
        if df2.shape[0] == 0:
            metricValue.append(np.nan)
            hitterVect.append(np.nan)
            gameVect.append(np.nan)
            dashboardVect.append(np.nan)
            compareToAverage.append(np.nan)
            dateType.append(np.nan)
            dateValue.append(np.nan)
            teamVect.append(np.nan)
        else:
            maxEV = df2.groupby('BatterId').quantile(0.9)['ExitSpeed'].mean()
            metricValue.append(maxEV)
            if df2['HomeTeam'].iloc[0] == "CLE_TIG":
                opponent = df2['AwayTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            else:
                opponent = df2['HomeTeam'].iloc[0]
                date = df2['GameID'].iloc[0][4:8]
                gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
            dateType.append('Month')
            dateValue.append(month_dict[month])
            hitter = df2['Batter']
            hitterVect.append(hitter.iloc[0])
            dashboardVect.append('Quality Contact')
            team = df2['BatterTeam']
            teamVect.append(team.iloc[0])
            if (maxEV >= avg_90 + std_90 * 1.5):
                compareToAverage.append(3)
            elif (maxEV <= avg_90 - std_90 * 1.5):
                compareToAverage.append(0)
            elif (maxEV <= avg_90 - std_90):
                compareToAverage.append(1)
            else:
                compareToAverage.append(2)
 

for bId in maxVelo_dict.keys():
    hitterIDVect.append(bId)
    df = data[data['BatterId'] == bId]  
    df = df[df['PitchCall'] == 'InPlay']
    metricName.append('90% EV')
    if df.shape[0] == 0:
        metricValue.append(np.nan)
        hitterVect.append(np.nan)
        gameVect.append(np.nan)
        dashboardVect.append(np.nan)
        compareToAverage.append(np.nan)
        dateType.append(np.nan)
        dateValue.append(np.nan)
        teamVect.append(np.nan)
    else:
        maxEV = df.groupby('BatterId').quantile(0.9)['ExitSpeed'].mean()
        metricValue.append(maxEV)
        if df['HomeTeam'].iloc[0] == "CLE_TIG":
            opponent = df['AwayTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        else:
            opponent = df['HomeTeam'].iloc[0]
            date = df['GameID'].iloc[0][4:8]
            gameVect.append('{}-{} {}'.format(date[0:2],date[2:4],opponent))
        dateType.append('Season')
        dateValue.append('2023')
        hitter = df['Batter']
        hitterVect.append(hitter.iloc[0])
        dashboardVect.append('Quality Contact')
        team = df['BatterTeam']
        teamVect.append(team.iloc[0])
        if (maxEV >= avg_90 + std_90 * 1.5):
            compareToAverage.append(3)
        elif (maxEV <= avg_90 - std_90 * 1.5):
            compareToAverage.append(0)
        elif (maxEV <= avg_90 - std_90):
            compareToAverage.append(1)
        else:
            compareToAverage.append(2)

In [24]:
# Preparing Columns for Export

#cols = ['Hitter','Game','Metric','Dashboard','Value','CompareToAverage', 'DateType', 'DateValue', 'Team']
gameAvgs = pd.DataFrame({'Hitter':hitterVect, 'Game':gameVect, 'Metric':metricName, 'Dashboard': dashboardVect, 'Value':metricValue, 'CompareToAverage':compareToAverage, 'DateType':dateType, 'DateValue':dateValue, 'Team':teamVect})

# Create DataFrame to merge back to offensive metrics
#wobaTable = pd.DataFrame({'HitterID':pitch_outcomes_season['BatterId'], 'wOBAVal':pitch_outcomes_season['wOBA'], 'PA':pitch_outcomes_season['ABs']})

#re24Table = pd.DataFrame({'HitterID':re24['BatterId'], 'RE24':re24['RE24']})

# Offensive Metrics
#metricsTable = pd.DataFrame({'Hitter':hitterVect, 'Game':gameVect, 'Metric':metricName, 'Dashboard': dashboardVect,
#                         'Value':metricValue, 'CompareToAverage':compareToAverage, 'DateType':dateType, 'DateValue':dateValue, 
#                         'Team':teamVect})

# Merge the Two
#gameAvgs = pd.merge(metricsTable, wobaTable, how = 'inner', on = 'HitterID')

#Remove all NA Rows
gameAvgs = gameAvgs.dropna()
gameAvgs = gameAvgs[gameAvgs.DateValue != 'Error']
#metricsTable = metricsTable.dropna()
#metricsTable = metricsTable[metricsTable.DateValue != 'Error']

# Remove Unneeeded Clemson Hitters 

#hittersRemoved = ['Byrd, Caid', 'Cunningham, J.P.', 'Farrisee, Patrick', 'Weiss, Willie', 'Lewis, David']  

#gameAvgs = gameAvgs[~gameAvgs.Hitter.isin(hittersRemoved)]

#Export File
#metricsTable.to_csv('../Work Files/NCAA Offensive KPIs by Week Month and Year Top 100 wOBA 2023.csv', index = False)
gameAvgs.to_excel('../Work Files/Clemson Offensive KPIs 2023 Regular Season.xlsx', index = False)

In [25]:
data['ZoneType'].value_counts()

Chase    2113
Waste    1815
Edge     1766
Heart     650
Name: ZoneType, dtype: int64

In [26]:
len(metricName)

8415